# NER

In [1]:
import polars as pl

def view_string(long_string, chunk_size=100):     
    return [long_string[i:i+chunk_size] for i in range(0, len(long_string), chunk_size)]

In [2]:
df = pl.read_csv('/home/sebacastillo/willow/output/sample.csv')

In [3]:
df.columns

['date_extract', 'date_article', 'content', 'link', 'authors']

In [4]:
text = df['content'][0]

In [5]:
view_string(text)

['9 Ago, 2023 La fiscal Silvia Bussano, de la UFI N°7 de Lanús, no tiene aún en sus manos el informe p',
 'reliminar de la autopsia, pero sí los forenses que analizaron el cuerpo de Morena Domínguez le adela',
 'ntaron sus conclusiones: la nena de 11 murió a causa de una hemorragia interna producto de una grave',
 ' lesión en el hígado. Así se lo confirmaron a Infobae fuentes de la investigación. “La autopsia reve',
 'ló que la nena falleció por un desgarro en el hígado producto del golpe profundo que recibió”, expli',
 'caron sobre los resultados de la pericia que se realizó en la Morgue Judicial de Lomas de Zamora. O ',
 'sea, los asesinos la piña en la panza que le dieron a la nena fue mortal. Por el crimen de Morena ha',
 'y dos detenidos: se trata de los hermanos Darío Humberto Madariaga, alias Lolo y de 25 años; y Migue',
 'l Ángel Madariaga, de 28.  Ambos están acusados del delito de “homicidio en ocasión de robo en concu',
 'rso real con encubrimiento”. Es que, fuentes del caso

In [6]:
len(text)

1091

# NER

In [18]:
from transformers import pipeline

nlp_ner = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    tokenizer=(
        'mrm8488/bert-spanish-cased-finetuned-ner',  
        {"use_fast": False}),
        aggregation_strategy="simple"
    )

t = 'Mis amigos están pensando viajar a Londres este verano'

nlp_ner(t)

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'LOC',
  'score': 0.99986696,
  'word': 'Londres',
  'start': None,
  'end': None}]

In [19]:
text_ner = nlp_ner(text)

In [20]:
type(text_ner)

list

In [21]:
len(text_ner)

15

In [22]:
text_ner[0:3]

[{'entity_group': 'PER',
  'score': 0.94463617,
  'word': 'Silvia Bussano',
  'start': None,
  'end': None},
 {'entity_group': 'ORG',
  'score': 0.94045573,
  'word': 'UFI',
  'start': None,
  'end': None},
 {'entity_group': 'LOC',
  'score': 0.8206464,
  'word': 'Lanús',
  'start': None,
  'end': None}]

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
custom_text_splitter = RecursiveCharacterTextSplitter(
    # Set custom chunk size
    chunk_size = 400,
    chunk_overlap  = 50,
    # Use length of the text as the size measure
    length_function = len,

)



In [28]:
# Create the chunks
texts = custom_text_splitter.create_documents([text])

In [29]:
texts

[Document(page_content='9 Ago, 2023 La fiscal Silvia Bussano, de la UFI N°7 de Lanús, no tiene aún en sus manos el informe preliminar de la autopsia, pero sí los forenses que analizaron el cuerpo de Morena Domínguez le adelantaron sus conclusiones: la nena de 11 murió a causa de una hemorragia interna producto de una grave lesión en el hígado. Así se lo confirmaron a Infobae fuentes de la investigación. “La autopsia', metadata={}),
 Document(page_content='Infobae fuentes de la investigación. “La autopsia reveló que la nena falleció por un desgarro en el hígado producto del golpe profundo que recibió”, explicaron sobre los resultados de la pericia que se realizó en la Morgue Judicial de Lomas de Zamora. O sea, los asesinos la piña en la panza que le dieron a la nena fue mortal. Por el crimen de Morena hay dos detenidos: se trata de los hermanos', metadata={}),
 Document(page_content='hay dos detenidos: se trata de los hermanos Darío Humberto Madariaga, alias Lolo y de 25 años; y Miguel 

In [30]:
type(texts)

list

# NER LONG DOCUMENTS

In [7]:
from transformers import pipeline

/home/sebacastillo/.willow/lib/python3.9/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [58]:
import spacy
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load Spanish stopwords from spacy
nlp = spacy.load("es_core_news_sm")
spanish_stopwords_spacy = spacy.lang.es.stop_words.STOP_WORDS

def remove_stopwords(text):
    """Remove Spanish stopwords from the text."""
    return " ".join([word for word in text.split() if word not in spanish_stopwords_spacy])

def langchain_chunk_text(text):
    
    # Remove stopwords
    # text = remove_stopwords(text)
    
    # Define a basic whitespace tokenizer
    def whitespace_tokenizer(text):
        return text.split()

    custom_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=60,
        length_function = len,
        #length_function=lambda x: len(whitespace_tokenizer(x))
    )
    
    # Create the chunks using langchain
    documents = custom_text_splitter.create_documents([text])
    
    # Extract the text content from the resulting Document objects
    chunks = [doc.page_content for doc in documents]
    
    return chunks


def ner_on_large_document(text):
    nlp_ner = pipeline(
        "ner",
        model="mrm8488/bert-spanish-cased-finetuned-ner",
        #model="PlanTL-GOB-ES/roberta-base-bne-capitel-ner",
        #tokenizer=('mrm8488/bert-spanish-cased-finetuned-ner', {"use_fast": False}),
        #aggregation_strategy="simple"
        aggregation_strategy="max" # more precision!
        #aggregation_strategy="average"
    )

    chunks = langchain_chunk_text(text)
    all_ner_results = []

    for chunk in chunks:
        ner_results = nlp_ner(chunk)
        all_ner_results.extend(ner_results)

    return all_ner_results




In [41]:
text

'9 Ago, 2023 La fiscal Silvia Bussano, de la UFI N°7 de Lanús, no tiene aún en sus manos el informe preliminar de la autopsia, pero sí los forenses que analizaron el cuerpo de Morena Domínguez le adelantaron sus conclusiones: la nena de 11 murió a causa de una hemorragia interna producto de una grave lesión en el hígado. Así se lo confirmaron a Infobae fuentes de la investigación. “La autopsia reveló que la nena falleció por un desgarro en el hígado producto del golpe profundo que recibió”, explicaron sobre los resultados de la pericia que se realizó en la Morgue Judicial de Lomas de Zamora. O sea, los asesinos la piña en la panza que le dieron a la nena fue mortal. Por el crimen de Morena hay dos detenidos: se trata de los hermanos Darío Humberto Madariaga, alias Lolo y de 25 años; y Miguel Ángel Madariaga, de 28.  Ambos están acusados del delito de “homicidio en ocasión de robo en concurso real con encubrimiento”. Es que, fuentes del caso adelantaron que la moto utilizada para atacar

In [59]:
# Test
results = ner_on_large_document(text)
print(results)

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'PER', 'score': 0.9996953, 'word': 'Silvia Bussano', 'start': 22, 'end': 36}, {'entity_group': 'ORG', 'score': 0.99941254, 'word': 'UFI', 'start': 44, 'end': 47}, {'entity_group': 'LOC', 'score': 0.9905323, 'word': 'Lanús', 'start': 55, 'end': 60}, {'entity_group': 'PER', 'score': 0.99846494, 'word': 'Morena Domínguez', 'start': 175, 'end': 191}, {'entity_group': 'ORG', 'score': 0.99982435, 'word': 'Infobae', 'start': 346, 'end': 353}, {'entity_group': 'ORG', 'score': 0.9993494, 'word': 'Infobae', 'start': 2, 'end': 9}, {'entity_group': 'LOC', 'score': 0.9903119, 'word': 'Morgue Judicial', 'start': 218, 'end': 233}, {'entity_group': 'LOC', 'score': 0.9991207, 'word': 'Lomas de Zamora', 'start': 237, 'end': 252}, {'entity_group': 'PER', 'score': 0.9555124, 'word': 'Morena', 'start': 347, 'end': 353}, {'entity_group': 'PER', 'score': 0.6737906, 'word': 'Morena', 'start': 3, 'end': 9}, {'entity_group': 'PER', 'score': 0.9993232, 'word': 'Darío Humberto Madariaga', 'start

In [60]:
import pprint

In [61]:
len(results)

14

In [62]:
pprint.pprint(results)

[{'end': 36,
  'entity_group': 'PER',
  'score': 0.9996953,
  'start': 22,
  'word': 'Silvia Bussano'},
 {'end': 47,
  'entity_group': 'ORG',
  'score': 0.99941254,
  'start': 44,
  'word': 'UFI'},
 {'end': 60,
  'entity_group': 'LOC',
  'score': 0.9905323,
  'start': 55,
  'word': 'Lanús'},
 {'end': 191,
  'entity_group': 'PER',
  'score': 0.99846494,
  'start': 175,
  'word': 'Morena Domínguez'},
 {'end': 353,
  'entity_group': 'ORG',
  'score': 0.99982435,
  'start': 346,
  'word': 'Infobae'},
 {'end': 9,
  'entity_group': 'ORG',
  'score': 0.9993494,
  'start': 2,
  'word': 'Infobae'},
 {'end': 233,
  'entity_group': 'LOC',
  'score': 0.9903119,
  'start': 218,
  'word': 'Morgue Judicial'},
 {'end': 252,
  'entity_group': 'LOC',
  'score': 0.9991207,
  'start': 237,
  'word': 'Lomas de Zamora'},
 {'end': 353,
  'entity_group': 'PER',
  'score': 0.9555124,
  'start': 347,
  'word': 'Morena'},
 {'end': 9,
  'entity_group': 'PER',
  'score': 0.6737906,
  'start': 3,
  'word': 'Morena'

# Traditional NER

In [ ]:
import spacy
from spacy.lang.es.examples import sentences 
doc = text
nlp = spacy.load("es_core_news_sm")
doc = nlp(doc)
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

In [67]:
for named_entity in doc.ents:
    if named_entity.label_ == "PER":
        print(named_entity)

Morena Domínguez
Así
Infobae
Darío Humberto Madariaga
Lolo
Miguel Ángel Madariaga
Morena


In [68]:
for named_entity in doc.ents:
    if named_entity.label_ == "LOC":
        print(named_entity)

Lanús
“
Morgue Judicial de Lomas de Zamora
O
“
Noticia


CONCLUSION: EL MODELO DE LENGUAJE TIENE MEJORES RESULTADO

In [69]:
ner_doc = pl.DataFrame(results)

In [70]:
ner_doc

entity_group,score,word,start,end
str,f64,str,i64,i64
"""PER""",0.999695,"""Silvia Bussano…",22,36
"""ORG""",0.999413,"""UFI""",44,47
"""LOC""",0.990532,"""Lanús""",55,60
"""PER""",0.998465,"""Morena Domíngu…",175,191
"""ORG""",0.999824,"""Infobae""",346,353
"""ORG""",0.999349,"""Infobae""",2,9
"""LOC""",0.990312,"""Morgue Judicia…",218,233
"""LOC""",0.999121,"""Lomas de Zamor…",237,252
"""PER""",0.955512,"""Morena""",347,353
